Demonstrations for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.core_expr_types.tuples</a>
========

In [ ]:
import proveit
from proveit import Lambda, ExprTuple, ExprRange, IndexedVar, safe_dummy_var, defaults
from proveit import A, a, b, f, i, j, k, l, m, n, x, y
from proveit.logic import InSet
#from proveit.logic.equality import multi_var_substitution
from proveit.numbers import Add, Mult, Neg, subtract, Natural, NaturalPos, Integer, zero, one, two
from proveit.core_expr_types import Len
from proveit.core_expr_types import (
    a_i_to_j, x_1_to_n, y_1_to_n, f_i_to_j, f_i_to_jp1)
from proveit.core_expr_types.tuples import (
        len_of_ranges_with_repeated_indices_from_1, range_from1_len,
        len_of_ranges_with_repeated_indices_from_1, len_of_ranges_with_repeated_indices,
        len_of_empty_range_of_ranges)
from proveit.core_expr_types.tuples import general_range_of_ranges
%begin demonstrations

In [ ]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, zero), one, n)).computation(assumptions=[InSet(n, NaturalPos)])

In [ ]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), two, n), one, zero)).computation(assumptions=[InSet(n, Natural)])

In [ ]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, n), one, n)).computation(assumptions=[InSet(n, Natural), InSet(n, NaturalPos)])

In [ ]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, zero), two, one)).computation(assumptions=[InSet(n, Natural)])

The following 5 cells have been commented out — they create and then depend on the one-element expr_range in the first cell, which returns an error

In [ ]:
# expr_range = ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, one), one, one)

In [ ]:
# lambda_map = Lambda((expr_range.parameter, expr_range.body.parameter), expr_range.body.body)

In [ ]:
# singular_nested_range_reduction.instantiate({f:lambda_map, i:one, j:one, m:one})

In [ ]:
# Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, one), one, one)).computation()

In [ ]:
# Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, two), one, one)).computation()

In [ ]:
assumptions = [InSet(Add(j, Neg(i), one), Natural), 
               InSet(i, Natural), InSet(j, Natural)]

In [ ]:
f_i_to_jp1

In [ ]:
eq = f_i_to_jp1.partition(j, assumptions=assumptions)

In [ ]:
eq.rhs.merger(assumptions=assumptions)

In [ ]:
f_i_to_j__shift__eq = \
    f_i_to_j.shift_equivalence(new_shift=one, assumptions=[InSet(subtract(Add(j, one), i),
                                                                 Natural)])

In [ ]:
f_i_to_j__shift__eq.rhs

In [ ]:
Add(subtract(i, one), one).simplification(assumptions=[InSet(i, Integer)])

In [ ]:
Add(Add(i, Neg(two)), two).simplification(assumptions=[InSet(i, Integer)])

In [ ]:
Add(subtract(j, one), one).simplification(assumptions=[InSet(j, Integer)])

In [ ]:
Add(subtract(j, two), two).simplification(assumptions=[InSet(j, Integer)])

In [ ]:
f_i_to_j__shift__eq.rhs.entries[0].shift_equivalence(
    old_shift=one, new_shift=two, 
    assumptions=[InSet(j, Integer), InSet(i, Integer),
                 InSet(subtract(j, subtract(i, one)), Natural)])

In [ ]:
tup=ExprTuple(a, b)

In [ ]:
Len(a_i_to_j).computation(assumptions=[InSet(Add(j, Neg(i), one), Natural)])

In [ ]:
tup_jdgmt = tup.prove(assumptions=[tup])

Not clear what the following cell is/was trying to accomplish:

In [ ]:
# tup.prove(assumptions=[tup]).instantiate({a:a_i_to_j},
#                             assumptions=[InSet(Add(j, Neg(i), one), Natural)])

In [ ]:
eq = f_i_to_jp1.partition(k, assumptions=[InSet(subtract(Add(k, one), i), Natural),
                              InSet(subtract(Add(j, one), k), Natural)])

In [ ]:
eq.rhs.merger(assumptions=[InSet(subtract(Add(k, one), i), Natural),
                              InSet(subtract(Add(j, one), k), Natural)])

In [ ]:
eq = f_i_to_jp1.partition(i, assumptions=[InSet(subtract(Add(j, one), i), Natural)])

In [ ]:
eq.rhs.merger(assumptions=[InSet(subtract(Add(j, one), i), Natural)])

In [ ]:
eq = f_i_to_j.partition(subtract(j, one), assumptions=[InSet(subtract(j, i), Natural)])

In [ ]:
Add(subtract(j, one), one).simplification(assumptions=[InSet(j, Natural)])

In [ ]:
eq.rhs.merger(assumptions=[InSet(subtract(j, i), Natural), InSet(j, Natural)])

In [ ]:
%end demonstrations